# Correspondance des organisations avec la base de données SIRENE

In [1]:
import os
os.chdir('/Users/Raphael/INRIA/travail/anHALytics/') #Select your working directory
cwd = os.getcwd()
import codecs
import pandas as pd
import os.path
from langdetect import detect_langs
import numpy as np
import matplotlib.pyplot as plt
accents=['à','á','â','ç','è','é','ê','ë','ì','í','î','ï','ñ','ò','ó','ô','ù','ú','û','ü','ý','ÿ']
spec=['/','*',',','.','?','-','_']
stopwords=['de','la','les','des','le','du']

### Récupération des données

#### Tables anHALytics

In [2]:
anHALytics={}
for x in os.listdir(cwd+'/json'):
    if (x!='ORG_SIREN') & (x!='.DS_Store'):
        anHALytics[x[:-5]]=pd.read_json(cwd+'/json/'+x)

#### Table SIRENE (un subset)

In [3]:
subset_SIRENE=pd.read_json('/Users/Raphael/INRIA/travail/SIRENE/subset_SIRENE.json')

### Fonction de normalisation du texte pour effectuer la correspondance

In [4]:
liste_mots=pd.read_csv('liste_mots.txt',sep='\t',encoding='"ISO-8859-1"')

mots_accent=[]
for x in list(liste_mots['mot']):
    if len(set(x).intersection(set(accents)))>0:
        mots_accent.append(x)

import unicodedata

dico_mot_accent={}

for x in mots_accent:
    for i,y in enumerate(x):
        if y in accents:
            break
    dico_mot_accent[unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8')]=x[:i]+x[i+1:]

In [17]:
def normalize(string,spec,stopwords,sigle=False): #pas très optimisé, mais le temps d'execution est plus que correct
    if string!=None:
        string=string.lower()
        if sigle==False:
            string=del_stopwords(string,stopwords)
            string=deal_accents(string,dico_mot_accent)
        for x in spec:
            string=string.replace(x,'')
        if sigle & (len(string)>3):
            return(string)
        if sigle & (len(string)<=3):
            return('')
        return(string.replace(' ',''))
    return('')

def deal_accents(string,dico_mot_accent):
    string=string.split(' ')
    for i,x in enumerate(string):
        if x in dico_mot_accent:
            string[i]=dico_mot_accent[x]
    return(' '.join(string))

def del_stopwords(string,stopwords):
    string=string.split(' ')
    for i in range(len(string)-1,-1,-1):
        if string[i] in stopwords:
            del string[i]
    return(' '.join(string))

### On normalise les colonnes correspondantes aux noms des entitées:

In [6]:
subset_SIRENE['recherche'] = subset_SIRENE['NOMEN_LONG'].apply(lambda x: normalize(x,spec,stopwords))

In [7]:
anHALytics['ORGANISATION_NAME']['recherche']=anHALytics['ORGANISATION_NAME']['name'].apply(lambda x: normalize(x,spec,stopwords))

La dernière étape consiste à joindre les deux tables via la colonne 'recherche':

In [8]:
merged=pd.merge(subset_SIRENE,anHALytics['ORGANISATION_NAME'],on='recherche')

Cependant, il reste quelques problèmes

In [9]:
table1=merged.groupby(['organisationID','name'],as_index=False).SIREN.count().sort_values(by='SIREN',ascending=False)
#table1.iloc[[0,8,9,12,14]].to_excel(cwd+'/Tables/'+'Table1.xls')
table1.head()



,organisationID,name,SIREN
752,15889,Institut National de la Sant?? et de la Recher...,166
868,18548,Institut National de la Sant?? et de la Recher...,166
47,510,Institut National de la Sant?? et de la Recher...,166
706,14582,Institut National de la Sant?? et de la Recher...,166
431,8559,Institut National de la Sant?? et de la Recher...,166


Premièrement, on remarque que des noms sont associés à plusieurs lignes de la base SIRENE. Ensuite que certaines organisations de la base anHALytics ont plusieurs lignes différentes.

In [10]:
table2=subset_SIRENE.groupby(["SIREN"],as_index=False).NIC.count().sort_values(by='NIC',ascending=False).head()
#table2.to_excel(cwd+'/Tables/'+'Table2.xls')
table2

,SIREN,NIC
116,180089013,848
98,180036048,166
115,180070039,165
56,130015506,98
351,199411117,73


On remarque que dans la table SIRENE, des codes SIREN apparaissent sur plusieurs lignes et ce malgré leur caractère unique. Les code SIREN sont bels et bien uniques, néanmoins chaque code est apparenté à une entreprise, or une entreprise peut possèder plusieurs établissements qui seront donc enregistrés dans la base de donnée. On se sert donc de l'attribut "SIEGE" pour ne laisser qu'une seul ligne pour chaque code SIREN, car une entreprise possède un et un seul siège:

In [11]:
subset_SIRENE=subset_SIRENE[subset_SIRENE['SIEGE']==1]
merged=pd.merge(subset_SIRENE[subset_SIRENE['recherche']!=''],
                anHALytics['ORGANISATION_NAME'][anHALytics['ORGANISATION_NAME']['recherche']!=''],
                on='recherche')

In [12]:
merged.groupby(['organisationID','name']).SIREN.count().sort_values(ascending=False).head()

organisationID  name
3373            3S      4
7662            SPI     3
6524            LSI     3
5719            IRIS    3
2996            PROG    3
Name: SIREN, dtype: int64

Le problème semble persister de manière plus légère, regardons de plus près celui-ci:

In [13]:
table3=merged[merged['organisationID']==2996][['organisationID','name','NOMEN_LONG','recherche','SIREN']]
#table3.to_excel(cwd+'/Tables/'+'Table3.xls')
table3

,organisationID,name,NOMEN_LONG,recherche,SIREN
289,2996,PROG,PRO-G,prog,389292178
291,2996,PROG,PROG,prog,453798944
293,2996,PROG,PRO-G,prog,327072880


On remarque sans suprise que les noms d'entreprise de sont pas "uniques", ou du moins les sigles ne le sont pas. Nous sommes donc dans des cas ambigüs, que je ne sais pas régler de manière automatique (pas assez d'information dans les autres champs de la base SIRENE), donc je choisi de ne pas inclure ces lignes dans le résultat final (le temps que je trouve une solution alternative)

In [14]:
orgid=merged.groupby(['organisationID']).SIREN.count()[merged.groupby(['organisationID']).SIREN.count()<2].index.values
org_SIREN=merged[merged['organisationID'].isin(orgid)][['organisationID','SIREN']]
org_SIREN.head()

,organisationID,SIREN
6,2214,821134491
7,9034,130018336
8,5323,130018351
9,14635,130018351
10,5864,130018484


## On effectue la même chose pour les SIGLE

In [18]:
subset_SIRENE['recherche_SIGLE'] = subset_SIRENE['SIGLE'].apply(lambda x: normalize(x,spec,stopwords,sigle=True))


anHALytics['ORGANISATION_NAME']['recherche_SIGLE']=anHALytics['ORGANISATION_NAME']['name'].apply(lambda x: normalize(x,spec,stopwords,sigle=True))


In [20]:
merged_SIGLE=pd.merge(subset_SIRENE[subset_SIRENE['recherche_SIGLE']!=''],
                anHALytics['ORGANISATION_NAME'][anHALytics['ORGANISATION_NAME']['recherche_SIGLE']!=''],
                on='recherche_SIGLE')

merged_SIGLE[['SIREN','NOMEN_LONG','SIGLE','recherche_SIGLE','name','organisationID']].head()

,SIREN,NOMEN_LONG,SIGLE,recherche_SIGLE,name,organisationID
0,338296593,ASS ECOLE SUPER INGEN GENIE ELECT TELEM,ESIGETEL,esigetel,ESIGETEL,9776
1,338296593,ASS ECOLE SUPER INGEN GENIE ELECT TELEM,ESIGETEL,esigetel,ESIGETEL,9777
2,338723919,EUROPEAN SYNCHROTRON RADIATION FACILITY,ESRF,esrf,ESRF,1969
3,338723919,EUROPEAN SYNCHROTRON RADIATION FACILITY,ESRF,esrf,ESRF,8219
4,338723919,EUROPEAN SYNCHROTRON RADIATION FACILITY,ESRF,esrf,ESRF,8220


In [21]:
orgid_SIGLE=merged_SIGLE.groupby(['organisationID']).SIREN.count()[merged_SIGLE.groupby(['organisationID']).SIREN.count()<2].index.values

org_SIREN_SIGLE=merged_SIGLE[merged_SIGLE['organisationID'].isin(orgid_SIGLE)][['organisationID','SIREN']]

org_SIREN=org_SIREN.append(org_SIREN_SIGLE)

In [22]:
org_SIREN=org_SIREN.drop_duplicates(['SIREN','organisationID'])

org_SIREN.index=[i for i in range(org_SIREN.shape[0])]

In [24]:
org_SIREN.head()

,organisationID,SIREN
0,2214,821134491
1,9034,130018336
2,5323,130018351
3,14635,130018351
4,5864,130018484


Cette nouvelle table permettra de relier la table organisation et la table SIRENE.